In [ ]:
import os
import sys
import json
import subprocess
import time
import glob
from pathlib import Path
from datetime import datetime
from collections import defaultdict

print("PHASE 1: SETUP AND DATA LOADING")
print()

WORK_DIR = os.getcwd()


DATA_ROOT = "/courses/CS5330.202610/data/FinalProject"
ANNOTATIONS_DIR = f"{DATA_ROOT}/annotations"
IMAGES_DIR = f"{DATA_ROOT}/images"
# ---------------------------------------------------

print("Installing dependencies...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q",
                       "ultralytics", "opencv-python", "pyyaml",
                       "--no-cache-dir"])

from ultralytics import YOLO
import cv2
import numpy as np
import torch

print("Dependencies installed successfully")
print()
print(f"Working directory: {WORK_DIR}")
print(f"Data root: {DATA_ROOT}")
print(f"Annotations directory: {ANNOTATIONS_DIR}")
print(f"Images directory: {IMAGES_DIR}")
print()

print("Loading dataset JSON files...")

try:
    with open(f"{ANNOTATIONS_DIR}/instances.json", 'r') as f:
        coco_data = json.load(f)
    print("Loaded instances.json")
except Exception as e:
    print(f"Error loading instances.json: {e}")
    raise

try:
    with open(f"{ANNOTATIONS_DIR}/instances_train.json", 'r') as f:
        train_data = json.load(f)
    print("Loaded instances_train.json")
except Exception as e:
    print(f"Error loading instances_train.json: {e}")
    raise

try:
    with open(f"{ANNOTATIONS_DIR}/instances_val.json", 'r') as f:
        val_data = json.load(f)
    print("Loaded instances_val.json")
except Exception as e:
    print(f"Error loading instances_val.json: {e}")
    raise

print()


train_img_ids = {img['id'] for img in train_data['images']}
val_img_ids = {img['id'] for img in val_data['images']}
images_dict = {img['id']: img for img in coco_data['images']}
categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
annotations = coco_data['annotations']

class_names = sorted([categories[c] for c in categories.keys()])

print()

print(f"  Training images: {len(train_img_ids)}")
print(f"  Validation images: {len(val_img_ids)}")
print(f"  Total images: {len(images_dict)}")
print(f"  Total annotations: {len(annotations)}")
print(f"  Number of classes: {len(class_names)}")
print(f"  Classes: {class_names}")
print()

missing_count = 0
for img_id, img_info in list(images_dict.items())[:5]:
    img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
    if not os.path.exists(img_path):
        missing_count += 1
        print(f"Missing: {img_path}")

if missing_count == 0:
    print("All checked images exist")
print()

print("PHASE 1 COMPLETE - Ready for Phase 2")
print()


print("PHASE 2: CONVERT TO YOLO FORMAT")
print()

import shutil

print("Creating directory structure...")

os.makedirs(f"{WORK_DIR}/dataset/images/train", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/images/val", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/labels/train", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/labels/val", exist_ok=True)

print("Directories created")
print()

def create_yolo_annotations(img_ids, split_name):
    processed = 0
    skipped = 0
    
    for img_id in img_ids:
        img_info = images_dict[img_id]
        img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
        
        if not os.path.exists(img_path):
            skipped += 1
            continue
        
        width, height = img_info['width'], img_info['height']
        
        img_annotations = [ann for ann in annotations if ann['image_id'] == img_id]
        
        txt_filename = Path(img_info['file_name']).stem + ".txt"
        txt_path = f"{WORK_DIR}/dataset/labels/{split_name}/{txt_filename}"
        
        with open(txt_path, 'w') as f:
            for ann in img_annotations:
                class_id = ann['category_id'] - 1
                bbox = ann['bbox']
                
                x_center = (bbox[0] + bbox[2] / 2) / width
                y_center = (bbox[1] + bbox[3] / 2) / height
                w_norm = bbox[2] / width
                h_norm = bbox[3] / height
                
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
        
        dst_img_path = f"{WORK_DIR}/dataset/images/{split_name}/{Path(img_info['file_name']).name}"
        if not os.path.exists(dst_img_path):
            shutil.copy(img_path, dst_img_path)
        
        processed += 1
    
    return processed, skipped

print("Converting training set...")
train_processed, train_skipped = create_yolo_annotations(train_img_ids, "train")
print(f"Training: {train_processed} processed, {train_skipped} skipped")
print()

print("Converting validation set...")
val_processed, val_skipped = create_yolo_annotations(val_img_ids, "val")
print(f"Validation: {val_processed} processed, {val_skipped} skipped")
print()

print("Creating YOLO data.yaml...")

yaml_content = f"""path: {WORK_DIR}/dataset
train: images/train
val: images/val

nc: {len(class_names)}
names: {class_names}
"""

with open(f"{WORK_DIR}/dataset/data.yaml", 'w') as f:
    f.write(yaml_content)

print("Created data.yaml")
print()

print("PHASE 2 COMPLETE - Ready for Phase 3")
print()


print("PHASE 3: TRAIN MODEL")
print()

print("Loading YOLOv8n model...")
model = YOLO('yolov8n.pt')
print("Model loaded")
print()

print("Checking available compute devices...")
cuda_available = torch.cuda.is_available()
device_str = "cuda:0" if cuda_available else "cpu"
print(f"CUDA available: {cuda_available}")
print(f"Using device: {device_str}")
print()

if not cuda_available:
    batch_size = 8
    print("Running on CPU - batch size set to 8")
else:
    batch_size = 16
    print("Running on GPU - batch size set to 16")
print()

print("Starting fine-tuning on Survue dataset...")
if not cuda_available:
    print("Training on CPU - estimated time: 1 to 1.5 hours")
else:
    print("Training on GPU - estimated time: 30 to 60 minutes")
print()

start_train = time.time()

results = model.train(
    data=f"{WORK_DIR}/dataset/data.yaml",
    epochs=50,
    imgsz=640,
    batch=batch_size,
    patience=10,
    device=device_str,
    project=f"{WORK_DIR}/runs",
    name="survue_yolov8n",
    save=True,
    verbose=False,
    workers=4
)

train_time = time.time() - start_train

print()
print(f"Training complete in {train_time/60:.2f} minutes")
print()

best_model_path = f"{WORK_DIR}/runs/survue_yolov8n/weights/best.pt"
if os.path.exists(best_model_path):
    model_size = os.path.getsize(best_model_path) / (1024 * 1024)
    print(f"Best model saved: {best_model_path}")
    print(f"Model size: {model_size:.2f} MB")
else:
    print(f"Error: Best model not found at {best_model_path}")
print()

print("PHASE 3 COMPLETE - Ready for Phase 4")
print()


print("PHASE 4: EVALUATE ON VALIDATION SET")
print()

run_dirs = glob.glob(f"{WORK_DIR}/runs/survue_yolov8n*")
run_dirs.sort(key=os.path.getmtime, reverse=True)
best_model_path = f"{run_dirs[0]}/weights/best.pt"
model = YOLO(best_model_path)
print(f"Loaded best model from {best_model_path}")
print()

print("Running inference on validation images...")
print("Using confidence threshold: 0.25")
print()

val_image_ids = list(val_img_ids)
detections_by_image = defaultdict(list)
ground_truth_by_image = defaultdict(list)

os.makedirs(f"{WORK_DIR}/results/predictions", exist_ok=True)

start_inference = time.time()

for idx, img_id in enumerate(val_image_ids):
    if idx % 50 == 0 or idx == len(val_image_ids) - 1:
        print(f"  Processing: {idx+1}/{len(val_image_ids)}")
    
    img_info = images_dict[img_id]
    img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
    
    if not os.path.exists(img_path):
        continue
    
    width, height = img_info['width'], img_info['height']
    
    img_gt = [ann for ann in annotations if ann['image_id'] == img_id]
    for ann in img_gt:
        bbox = ann['bbox']
        ground_truth_by_image[img_id].append({
            'class_id': ann['category_id'] - 1,
            'bbox': [bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]],
            'area': ann['area']
        })
    
    results = model.predict(source=img_path, conf=0.25, device=device_str, verbose=False)
    result = results[0]
    
    if result.boxes is not None:
        boxes = result.boxes
        for i in range(len(boxes)):
            box = boxes.xyxy[i].cpu().numpy()
            conf = float(boxes.conf[i])
            class_id = int(boxes.cls[i])
            
            detections_by_image[img_id].append({
                'class_id': class_id,
                'bbox': box.tolist(),
                'confidence': conf
            })
    
    if idx % 10 == 0:
        annotated = result.plot()
        output_path = f"{WORK_DIR}/results/predictions/{idx:04d}_pred.jpg"
        cv2.imwrite(output_path, annotated)

inference_time = time.time() - start_inference
avg_inference_time = inference_time / len(val_image_ids)

print()
print(f"Inference complete")
print(f"Total time: {inference_time:.2f}s")
print(f"Average per image: {avg_inference_time:.4f}s ({1/avg_inference_time:.2f} FPS)")
print()

print("PHASE 4 COMPLETE - Ready for Phase 5")
print()


print("PHASE 5: CALCULATE METRICS")
print()

def calculate_iou(box1, box2):
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])
    
    if x2_inter < x1_inter or y2_inter < y1_inter:
        return 0.0
    
    inter_area = (x2_inter - x1_inter) * (y2_inter - y1_inter)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0.0

def calculate_metrics(detections_by_image, ground_truth_by_image, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    
    for img_id in ground_truth_by_image:
        gt = ground_truth_by_image[img_id]
        dets = detections_by_image.get(img_id, [])
        
        sorted_dets = sorted(dets, key=lambda x: x['confidence'], reverse=True)
        gt_matched = [False] * len(gt)
        
        for det in sorted_dets:
            max_iou = 0
            max_gt_idx = -1
            
            for gt_idx, gt_box in enumerate(gt):
                if gt_matched[gt_idx]:
                    continue
                
                iou = calculate_iou(det['bbox'], gt_box['bbox'])
                if iou > max_iou:
                    max_iou = iou
                    max_gt_idx = gt_idx
            
            if max_iou >= iou_threshold and max_gt_idx >= 0:
                tp += 1
                gt_matched[max_gt_idx] = True
            else:
                fp += 1
        
        fn += sum(1 for matched in gt_matched if not matched)
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1, tp, fp, fn

print("Calculating metrics with IoU threshold at 0.5...")
precision, recall, f1, tp, fp, fn = calculate_metrics(detections_by_image, ground_truth_by_image)

total_gt = sum(len(gt) for gt in ground_truth_by_image.values())
total_detections = sum(len(dets) for dets in detections_by_image.values())

print()
print("EVALUATION RESULTS:")
print()
print(f"True Positives:      {tp}")
print(f"False Positives:     {fp}")
print(f"False Negatives:     {fn}")
print(f"Total Detections:    {total_detections}")
print(f"Total Ground Truth:  {total_gt}")
print()
print(f"Precision:           {precision:.4f}")
print(f"Recall:              {recall:.4f}")
print(f"F1-Score:            {f1:.4f}")
print()

run_dirs = glob.glob(f"{WORK_DIR}/runs/survue_yolov8n*")
run_dirs.sort(key=os.path.getmtime, reverse=True)
best_model_path = f"{run_dirs[0]}/weights/best.pt"
model_size = os.path.getsize(best_model_path) / (1024 * 1024)
print("MODEL CHARACTERISTICS:")
print()
print(f"Model Size:          {model_size:.2f} MB")
print(f"Inference Time:      {avg_inference_time:.4f}s per image")
print(f"Speed:               {1/avg_inference_time:.2f} FPS")
print()

print("HARDWARE SUITABILITY FOR BIKE MOUNT:")
print()
lightweight = model_size < 50
fast = avg_inference_time < 0.1
accurate = precision > 0.7 and recall > 0.7

print(f"Lightweight (under 50MB):     {lightweight} ({model_size:.2f} MB)")
print(f"Fast (under 0.1s per image):  {fast} ({avg_inference_time:.4f}s)")
print(f"Accurate (P>0.7, R>0.7):      {accurate} (P:{precision:.4f}, R:{recall:.4f})")
print()

print("Saving metrics to JSON...")

metrics_data = {
    'timestamp': datetime.now().isoformat(),
    'model_info': {
        'architecture': 'YOLOv8n',
        'training_approach': 'Fine-tuned on Survue dataset',
        'model_size_mb': float(model_size),
        'inference_time_per_image': float(avg_inference_time),
        'fps': float(1 / avg_inference_time),
    },
    'training_config': {
        'epochs': 50,
        'batch_size': batch_size,
        'image_size': 640,
        'patience': 10,
        'device': device_str,
    },
    'dataset': {
        'training_images': train_processed,
        'validation_images': val_processed,
        'classes': class_names,
        'total_annotations': total_gt,
    },
    'evaluation_results': {
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'true_positives': int(tp),
        'false_positives': int(fp),
        'false_negatives': int(fn),
        'total_ground_truth': int(total_gt),
        'total_detections': int(total_detections),
        'iou_threshold': 0.5,
        'confidence_threshold': 0.25,
    },
    'hardware_suitability': {
        'lightweight': lightweight,
        'fast': fast,
        'accurate': accurate,
    }
}

os.makedirs(f"{WORK_DIR}/results", exist_ok=True)
with open(f"{WORK_DIR}/results/evaluation_metrics.json", 'w') as f:
    json.dump(metrics_data, f, indent=4)

print("Metrics saved to results/evaluation_metrics.json")
print()

print("ALL PHASES COMPLETE")

PHASE 1: SETUP AND DATA LOADING

Installing dependencies...



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Dependencies installed successfully

Working directory: /courses/CS5330.202610/students/reddy.ada/runs/detect/val9
Data root: /courses/CS5330.202610/data/FinalProject
Annotations directory: /courses/CS5330.202610/data/FinalProject/annotations
Images directory: /courses/CS5330.202610/data/FinalProject/images

Loading dataset JSON files...
Loaded instances.json
Loaded instances_train.json
Loaded instances_val.json

Creating dataset mappings...

Dataset Statistics:
  Training images: 393
  Validation images: 107
  Total images: 500
  Total annotations: 9787
  Number of classes: 3
  Classes: ['human', 'trafficsign', 'vehicle']

Verifying image files...
All checked images exist

PHASE 1 COMPLETE - Ready for Phase 2

PHASE 2: CONVERT TO YOLO FORMAT

Creating directory structure...
Directories created

Converting training set...


In [8]:
import os
import sys
import json
import shutil
import time
from pathlib import Path
from collections import defaultdict
from datetime import datetime
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

# --------------------------
# PHASE 0: Paths and Setup
# --------------------------
WORK_DIR = os.getcwd()
DATA_ROOT = "/courses/CS5330.202610/data/FinalProject"
ANNOTATIONS_DIR = f"{DATA_ROOT}/annotations"
IMAGES_DIR = f"{DATA_ROOT}/images"

# Install dependencies if not already
import subprocess
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "ultralytics", "opencv-python", "--no-cache-dir"])

from ultralytics import YOLO
import cv2
import torch
import numpy as np

os.makedirs(f"{WORK_DIR}/dataset/images/train", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/images/val", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/labels/train", exist_ok=True)
os.makedirs(f"{WORK_DIR}/dataset/labels/val", exist_ok=True)
os.makedirs(f"{WORK_DIR}/results/predictions", exist_ok=True)

# --------------------------
# PHASE 1: Load COCO JSON
# --------------------------
with open(f"{ANNOTATIONS_DIR}/instances.json", "r") as f:
    coco_data = json.load(f)

with open(f"{ANNOTATIONS_DIR}/instances_train.json", "r") as f:
    train_data = json.load(f)

with open(f"{ANNOTATIONS_DIR}/instances_val.json", "r") as f:
    val_data = json.load(f)

train_img_ids = {img['id'] for img in train_data['images']}
val_img_ids = {img['id'] for img in val_data['images']}
images_dict = {img['id']: img for img in coco_data['images']}
categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
annotations = coco_data['annotations']

class_names = sorted([categories[c] for c in categories.keys()])

# --------------------------
# PHASE 2: Convert JSON → YOLO TXT
# --------------------------
def create_yolo_labels(img_ids, split_name):
    processed = 0
    skipped = 0
    for img_id in img_ids:
        img_info = images_dict[img_id]
        img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
        if not os.path.exists(img_path):
            skipped += 1
            continue

        width, height = img_info['width'], img_info['height']
        img_annotations = [ann for ann in annotations if ann['image_id'] == img_id]

        # Create label txt
        txt_filename = Path(img_info['file_name']).stem + ".txt"
        txt_path = f"{WORK_DIR}/dataset/labels/{split_name}/{txt_filename}"
        with open(txt_path, "w") as f:
            for ann in img_annotations:
                class_id = ann['category_id'] - 1
                bbox = ann['bbox']
                x_center = (bbox[0] + bbox[2]/2)/width
                y_center = (bbox[1] + bbox[3]/2)/height
                w_norm = bbox[2]/width
                h_norm = bbox[3]/height
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

        # Copy image to dataset folder
        dst_img_path = f"{WORK_DIR}/dataset/images/{split_name}/{Path(img_info['file_name']).name}"
        if not os.path.exists(dst_img_path):
            shutil.copy(img_path, dst_img_path)

        processed += 1

    return processed, skipped

train_processed, train_skipped = create_yolo_labels(train_img_ids, "train")
val_processed, val_skipped = create_yolo_labels(val_img_ids, "val")

# --------------------------
# PHASE 3: dataset.yaml
# --------------------------
yaml_content = f"""
path: {WORK_DIR}/dataset
train: images/train
val: images/val

nc: {len(class_names)}
names: {class_names}
"""

with open(f"{WORK_DIR}/dataset/data.yaml", "w") as f:
    f.write(yaml_content)

# --------------------------
# PHASE 4: YOLOv8 Micro Model
# --------------------------
micro_model_yaml = f"""
# YOLOv8-Micro

nc: {len(class_names)}
depth_multiple: 0.33
width_multiple: 0.25

backbone:
  - [-1, 1, Conv, [16, 3, 2]]
  - [-1, 1, Conv, [32, 3, 2]]
  - [-1, 1, C2f, [32, 1]]
  - [-1, 1, Conv, [64, 3, 2]]
  - [-1, 1, C2f, [64, 1]]

head:
  - [-1, 1, Conv, [64, 3, 1]]
  - [[-1], 1, Detect, [nc]]
"""

with open(f"{WORK_DIR}/dataset/micro_model.yaml", "w") as f:
    f.write(micro_model_yaml)

# Load micro model
model = YOLO("yolov8n.pt")  # Pretrained weights as base
print("Model loaded")

# --------------------------
# PHASE 5: Train
# --------------------------
device_str = "cuda:0" if torch.cuda.is_available() else "cpu"
batch_size = 16 if torch.cuda.is_available() else 8

start_train = time.time()
results = model.train(
    data=f"{WORK_DIR}/dataset/data.yaml",
    model=f"{WORK_DIR}/dataset/micro_model.yaml",
    epochs=50,
    imgsz=640,
    batch=batch_size,
    patience=10,
    device=device_str,
    project=f"{WORK_DIR}/runs",
    name="survue_yolov8m",
    save=True,
    workers=4
)
train_time = time.time() - start_train
print(f"Training complete in {train_time/60:.2f} minutes")

# --------------------------
# PHASE 6: Evaluate
# --------------------------
best_model_path = f"{WORK_DIR}/runs/survue_yolov8m/weights/best.pt"
model = YOLO(best_model_path)

val_image_ids = list(val_img_ids)
detections_by_image = defaultdict(list)
ground_truth_by_image = defaultdict(list)

start_inference = time.time()
for idx, img_id in enumerate(val_image_ids):
    img_info = images_dict[img_id]
    img_path = f"{IMAGES_DIR}/{img_info['file_name']}"
    if not os.path.exists(img_path):
        continue

    width, height = img_info['width'], img_info['height']

    # ground truth
    img_gt = [ann for ann in annotations if ann['image_id'] == img_id]
    for ann in img_gt:
        bbox = ann['bbox']
        ground_truth_by_image[img_id].append({
            "class_id": ann['category_id']-1,
            "bbox": [bbox[0], bbox[1], bbox[0]+bbox[2], bbox[1]+bbox[3]],
            "area": ann['area']
        })

    results = model.predict(source=img_path, conf=0.25, device=device_str, verbose=False)
    result = results[0]

    if result.boxes is not None:
        boxes = result.boxes
        for i in range(len(boxes)):
            box = boxes.xyxy[i].cpu().numpy()
            conf = float(boxes.conf[i])
            class_id = int(boxes.cls[i])
            detections_by_image[img_id].append({
                "class_id": class_id,
                "bbox": box.tolist(),
                "confidence": conf
            })

    if idx % 10 == 0:
        annotated = result.plot()
        output_path = f"{WORK_DIR}/results/predictions/{idx:04d}_pred.jpg"
        cv2.imwrite(output_path, annotated)

avg_inference_time = (time.time() - start_inference)/len(val_image_ids)

# --------------------------
# PHASE 7: Metrics
# --------------------------
def calculate_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    if x2<x1 or y2<y1: return 0.0
    inter = (x2-x1)*(y2-y1)
    union = (box1[2]-box1[0])*(box1[3]-box1[1]) + (box2[2]-box2[0])*(box2[3]-box2[1]) - inter
    return inter/union if union>0 else 0.0

def calculate_metrics(dets, gt, iou_th=0.5):
    tp, fp, fn = 0,0,0
    for img_id in gt:
        g = gt[img_id]
        d = dets.get(img_id, [])
        g_matched = [False]*len(g)
        d_sorted = sorted(d, key=lambda x: x['confidence'], reverse=True)
        for det in d_sorted:
            max_iou = 0
            max_idx = -1
            for idx, box in enumerate(g):
                if g_matched[idx]: continue
                iou = calculate_iou(det['bbox'], box['bbox'])
                if iou>max_iou:
                    max_iou=iou
                    max_idx=idx
            if max_iou>=iou_th and max_idx>=0:
                tp+=1
                g_matched[max_idx]=True
            else:
                fp+=1
        fn+=sum(1 for m in g_matched if not m)
    precision = tp/(tp+fp) if (tp+fp)>0 else 0
    recall = tp/(tp+fn) if (tp+fn)>0 else 0
    f1 = 2*precision*recall/(precision+recall) if (precision+recall)>0 else 0
    return precision, recall, f1, tp, fp, fn

precision, recall, f1, tp, fp, fn = calculate_metrics(detections_by_image, ground_truth_by_image)
total_gt = sum(len(v) for v in ground_truth_by_image.values())
total_det = sum(len(v) for v in detections_by_image.values())

# Save metrics
metrics = {
    "precision": precision,
    "recall": recall,
    "f1": f1,
    "tp": tp,
    "fp": fp,
    "fn": fn,
    "total_gt": total_gt,
    "total_det": total_det,
    "avg_inference_time": avg_inference_time,
}

with open(f"{WORK_DIR}/results/evaluation_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

print("✅ Training and evaluation complete")
print(f"Metrics saved to {WORK_DIR}/results/evaluation_metrics.json")
print(f"Inference images saved to {WORK_DIR}/results/predictions/")

# --------------------------
# PHASE 7: Metrics Display (FINAL FIXED VERSION)
# --------------------------

import os

# --- Model Size ---
best_model_path = f"{WORK_DIR}/runs/survue_yolov8m/weights/best.pt"
model_size_mb = os.path.getsize(best_model_path) / (1024 * 1024)


# --- IoU function ---
def iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    area1 = max(0, box1[2] - box1[0]) * max(0, box1[3] - box1[1])
    area2 = max(0, box2[2] - box2[0]) * max(0, box2[3] - box2[1])
    union = area1 + area2 - inter + 1e-9

    return inter / union if union > 0 else 0

# --- Core PR metrics ---
precision, recall, f1, tp, fp, fn = calculate_metrics(
    detections_by_image, ground_truth_by_image
)

total_gt = sum(len(v) for v in ground_truth_by_image.values())
total_det = sum(len(v) for v in detections_by_image.values())
avg_inference_time = (time.time() - start_inference) / len(val_image_ids)

# --- mAP@50 ---
correct = 0
total_pred = 0
iou_threshold = 0.5

for img_id, preds in detections_by_image.items():
    gts = ground_truth_by_image.get(img_id, [])
    for pred in preds:
        total_pred += 1
        if any(iou(pred["bbox"], gt["bbox"]) >= iou_threshold for gt in gts):
            correct += 1

map50 = correct / total_pred if total_pred > 0 else 0

# --------------------------
# Save + Display Metrics
# --------------------------

metrics = {
    "Precision": precision,
    "Recall": recall,
    "F1-score": f1,
    "True Positives": tp,
    "False Positives": fp,
    "False Negatives": fn,
    "Total Ground Truth": total_gt,
    "Total Detections": total_det,
    "Average Inference Time (s/image)": avg_inference_time,
    "mAP@50": map50,
    "Model Size (MB)": model_size_mb,
}

# Save
os.makedirs(f"{WORK_DIR}/results", exist_ok=True)
with open(f"{WORK_DIR}/results/evaluation_metrics.json", "w") as f:
    json.dump(metrics, f, indent=4)

# Print nicely
print("\n================== Evaluation Metrics ==================")
for k, v in metrics.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}")
    else:
        print(f"{k}: {v}")
print("========================================================\n")
print(f"✅ Metrics saved to {WORK_DIR}/results/evaluation_metrics.json")
print(f"✅ Inference images saved to {WORK_DIR}/results/predictions/")



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


Model loaded
New https://pypi.org/project/ultralytics/8.3.235 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.231 🚀 Python-3.10.16 torch-2.5.1 CUDA:0 (Tesla P100-PCIE-12GB, 12194MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/courses/CS5330.202610/students/reddy.ada/runs/detect/val9/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/courses/CS5330.202610/students/redd

In [7]:
best_model_path = f"{WORK_DIR}/runs/survue_yolov8m/weights/best.pt"
model_size_mb = os.path.getsize(best_model_path) / (1024 * 1024)
print("Model Size (MB):", model_size_mb)

Model Size (MB): 5.936800956726074
